# Que peut-on dire des statistiques des délits et des crimes à partir des établissements de boucheries-charcutiers ?

L'objectif de cette étude est de pouvoir savoir s'il y a une corrélation, voire une causalité entre le nombre de boucheries-charcutiers d'une zone géographique et le nombre de délits et crimes reportés. Pour cela, nous utilisons principalement des sources statistiques issues du système statistique publique français.

En premier lieu, nous utilisons les statistiques du ministère de l'intérieur en ce qui concernent les délits et les crimes disponible en OpenData sur data.gouv.fr (https://www.data.gouv.fr/fr/datasets/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012/)
Puis, pour obteniir les établissements de boucheries-charcutiers, nous utilisons le répertoire Sirene, via le site sirene.fr.
Enfin, nous utilisons les populations légales pour contrôler l'effet taille des zones géographiques (communes, départements) et d'autres variables socio-économiques susceptibles d'être corrélées avec notre variable d'intérêt (taux de chômage, revenu localisé).

Nous essaierons de suivre une démarche GSBPM (https://statswiki.unece.org/display/GSBPM/Clickable+GSBPM+v5.1).

L'étude portera principalement au niveau département, une extension au niveau communal pourra également être faite.


## 1 - Récupération des données au niveau département (3-5 build, collect and process)

Les données de la statistique publique français sont accessible par Internet soit par site web (scrapping ou téléchargement d'une archive/fichier), soit par API. La majeure partie des cas est soit le téléchargement d'un fichier dont le format d'interopérabilité est variable (https://www.numerique.gouv.fr/uploads/Referentiel_General_Interoperabilite_V2.pdf), soit le téléchargement via requetage d'une API.

Voici le tableau des sources utilisées :

| source  | description  | producteur  | millésime  | formats  | champ |
|---|---|---|---|---|---|
| SIRENE   | Nombre de boucheries-charcutier  |  INSEE | courant  |  csv | France entière  |
|  GN/PN   | Nombre de crimes et délits rapportés  | Ministère de l'intérieur  | 2018   |  xls | France entière sauf Mayotte + Saint Martin(COM) |
|  Filosofi  | Revenu localisé   | INSEE  | 2018  | zip-csv  | France métro + Réunion + Martinique |
| Population légales  | Nombre d'habitants par communes/départements  | INSEE  |  2018 |  zip-csv | France entière sauf Mayotte |
| ESTEL   | Taux de chômage et d'emploi localisé  | INSEE  | 2018  |  xls |  France entière sauf Mayotte |
| SIASP  |  Nombre de gendarmes/policiers  | INSEE  | 2021  | xls  | France entière sauf Mayotte |

Compte tenu du foisonnement des données disponibles et le manque de données pour certains millésimes, nous nous sommes focalisés sur un millésime particulier 2018. Les dernières populations légales disponibles au début du projet concernait le millésime 2018. Nous avons donc pris comme point de référence cette année.


Le champ de l'étude a été déduit par intersection des champs des sources disponibles. Quelques rares sources présentaient des données des COM, d'autres ne présentaient pas parmi les DOM, des données pour Mayotte, devenu une DOM en 2011 où les sources statistiques n'ont pas encore atteint une maturité suffisante. Par manque de données, l'étude s'est restreinte au niveau France métropolitaine.
Egalement, par simplification, nous considérons les données récentes comme des données 2018 sans appliquer de vieillissement statistique, comme ce qui pourrait se faire dans le modèle Ines avec la source ERFS.

Les données du ministère de l'intérieur ont été mis en forme à partir du fichier xls qui présentait les totaux par commissariat et gendarmerie, pour passer à des statistiques départementaux. Les données de Sirene ont été agrégées du niveau individuel (établissement/ SIRET) au niveau département. Les autres sources présentaient déjà des statistiques prêtes à l'emploi au niveau département.

Nous proposons des statistiques rapportés à la population du département pour prendre en compte l'effet taille pour les variables d'intérêt concernées.


In [ ]:
%run "./Etude départementale/1_Recup_donnees_departementales.ipynb"

## 2 - Analyse exploratoire des données départementales (6 Analyse)

### 2 - A - Présentation des variables 

Nous nous intéressons aux nombres de crimes et délits pour les données de Ministère de l'Intérieur et au nombre de boucheries-charcutiers
Pour les sources auxiliaires, nous sélectionnons les variables socio-économiques des départements, à savoir le taux de pauvreté, le niveau de vie (médian, 1er décile, 9ème décile, rapport interdécile) et le taux de chômage.




| variable | description  |  source | type | Remarque |
|---|---|---|---|---|
| TP6018   | Taux de pauvreté monétaire (au seuil de 60% du niveau de vie médian) -Ensemble (%)  | Filosofi-INSEE  | Quantitatif-Niveau   ||
| MED18   | Médiane du niveau vie (€)  | Filosofi-INSEE  | Quantitatif-Niveau  ||
| D118   | 1er décile du niveau de vie (€)  | Filosofi-INSEE  | Quantitatif-Niveau  | Optionnel si étude de RD18|
| D918   | 9e décile du niveau de vie (€)   | Filosofi-INSEE  | Quantitatif-Niveau  | Optionnel si étude de RD18|
| RD18   | Rapport interdécile 9e décile/1er décile  | Filosofi-INSEE  |  Quantitatif-Ratio ||
| T1_2018   |  Estimation du taux de chômage localisé (%) | ESTEL-INSEE  | Quantitatif-Proportion  ||
| Nb_PN_GN_dep_100k_hab   | Nombre de policiers et gendarmes pour 100 000 habitants  | SIASP-INSEE  | Quantitatif-Proportion  ||
| Crim_Del_PN_GN   | Nombre de crimes et délits rapportés par l'ensemble des gendarmeries et commissariats  | Ministère de l'intérieur  | Quantitatif-Total  ||
| Nb_Boucherie_dep_hab   | Nombre de boucheries-charcutiers par habitant   | Sirene-INSEE  | Quantitatif-Proportion  ||
| Crim_Del_GN_hab   | Nombre de crimes et délits par habitant rapportés par l'ensemble des gendarmeries | Ministère de l'intérieur  | Quantitatif-Proportion  ||
| Crim_Del_PN_hab   | Nombre de crimes et délits par habitant rapportés par l'ensemble des commissariats   | Ministère de l'intérieur  | Quantitatif-Proportion  ||
| Crim_Del_PN_GN_hab   | Nombre de crimes et délits par habitant rapportés par l'ensemble des gendarmeries et commissariats  | Ministère de l'intérieur  | Quantitatif-Proportion  ||

### 2 - B - Statistiques univariées et bivariées

Départements outliers :

|TP6018|MED18|RD18|T1_2018|Nb_Boucherie_dep_hab|Nb_PN_GN_dep_100k_hab|Crim_Del_PN_GN_hab|
|---|---|---|---|---|---|---|
|66|74|74|66|2A|19|13|
|93|75|75||48|2A|69|
||78|92|||52|75|
||92|94|||75|92|
||93|||||93|


Le nombre de crimes et délits pour 100 000 habitants est atypique pour Paris (13052 contre la médiane 4270) et également pour les départements suivants Hauts-de-Seine (6758), Seine-Saint-Denis (8640), Bouche-du-Rhone (7469), Rhone (7380)


Le nombre de crimes et délits semble peu corrélé avec le nombre de boucherie, alors qu'il est fortement corrélé avec le rapport interdécile


In [ ]:
# Pensez à faire dans un terminal :
# conda install geoviews
%run "./Etude départementale/2_Cartes_descriptives_departementale.ipynb"

## 3 - Modélisation des données départementales (6 Analyse)

Dans un premier temps, nous étudions à l'aide d'une régression linéaire multiple, la pertinence du modèle, en étudiant dans un premier temps la corrélation entre le nombre de crimes et délits total à ses regresseurs, dont particulièrement le nombre de boucheries-charcutiers
Dans un second temps, nous utilisons un SVM, non pas pour classifier les observations mais pour effectuer une régression linéaire à vecteur de support

### 3 - A - Modélisation économétrique

### 3 - B - Approche machine learning

In [ ]:
%run "./Etude départementale/3_Modelisation_departementale.ipynb"

## 4 - Extension au niveau communal

### 4 - A  - Récupération des données au niveau communal (3-5 build, collect and process)


Nous utilisons les mêmes sources que précedemment, en ajoutant le référentiel des communes (COG), millésime 2021. De nouveau, nous faisons l'hypothèse simplificatrice de nous placer en géographie 2021 pour des données qui concernent en majeur partie des communes présentes en 2018. Nous utilisons également la localisation des gendarmeries et commissariats, disponibles en OpenData.

| source  | description  | producteur  | millésime  | formats  | champ |
|---|---|---|---|---|---|
| SIRENE   | Nombre de boucheries-charcutier  |  INSEE | courant  |  csv | France entière  |
|  GN/PN   | Nombre de crimes et délits rapportés  | Ministère de l'intérieur  | 2018   |  xls | France entière sauf Mayotte + Saint Martin(COM) |
|  Filosofi  | Revenu localisé   | INSEE  | 2018  | zip-csv  | France métro + Réunion + Martinique |
| Population légales  | Nombre d'habitants par communes/départements  | INSEE  |  2018 |  zip-csv | France entière sauf Mayotte |
| ESTEL   | Taux de chômage et d'emploi localisé  | INSEE  | 2018  |  xls |  France entière sauf Mayotte |
| SIASP  |  Nombre de gendarmes/policiers  | INSEE  | 2019  | xls  | France entière sauf Mayotte |
| **COG**  |  **Référentiel géographique des communes** | **INSEE**  | **2021**  | **zip-csv**  | **France entière** |
|**PN**| **Liste des services de police accueillant du public avec géolocalisation**|**Ministère de l'intérieur**|**(maj 2018)**|**csv**|**France entière**|
|**GN**| **Liste des unités de gendarmerie accueillant du public, comprenant leur géolocalisation et leurs horaires d'ouverture**|**Ministère de l'intérieur**|**(maj 2018)**|**csv**|**France entière**|

Concernant les données du ministère de l'intérieur, les crimes et délits sont rapport par commissariat et autres juridictions de la police nationale, et les gendarmeries. Les libellés des commissariats et des gendarmeries comportent le nom des communes. Cependant, il n'y a pas d'identifiant unique entre les données et les métadonnées de géolocalisation. Pour les organismes autres que les commissariats et les gendarmeries, le parti pris a été de ne pas les exploiter dans un premier temps (constituant une piste d'amélioration) car difficilement géolocalisable.

Pour apparier données et métadonnées de géolocalisation, nous identifions d'abord la commune du commissariat ou de la gendarmerie dans le COG, puis nous cherchons cette commune dans les métadonnées de géolocalisation. L'algorithme d'identification de la commune s'appuie sur la librairie python fuzzywuzzy, calculant des distances de Levenshtein avec différentes stratégies. Une reprise manuelle de certains appariemments a été nécessaire.

Après appariements avec la commune associés, certains commissariats et gendarmeries ne sont pas présents dans le fichier de métadonnées de géolocalisation, néanmoins nous disposons de l'adresse et nous interrogeons l'API api-adresse.data.gouv.fr pour obtenir à partir de l'adresse postale, les coordonnées x,y.

A partir des coordonnées géolocalisés, nous souhaitons ventiler les totaux aux alentours, en utilisant un proxy a priori de sentiment de sécurité.


In [ ]:
%run "./Etude communale/1_Recup_donnees_communales.ipynb"